In [1]:
import importlib
import numpy as np

from constants import *
import utils
importlib.reload(utils)

<module 'utils' from '/mnt/cb8c2a81-32e7-40bc-af74-0c1386cda683/programowanie/pipelon/Classifier-Builder/utils.py'>

In [2]:
x_train, y_train = utils.new_load_data('training')
x_test, y_test = utils.new_load_data('testing')

x_train, y_train = x_train[:10000], y_train[:10000]
x_test, y_test = x_test[:3000], y_test[:3000]

x_train.shape, y_test.shape

((10000, 20), (3000,))

In [3]:
print(f"Classes: {STATES}")
print(f"Class distribution for train: {np.unique(y_train, return_counts=True)}")
print(f"Class distribution for test: {np.unique(y_test, return_counts=True)}")

Classes: ['boredom', 'neutral']
Class distribution for train: (array([1., 2.]), array([4969, 5031]))
Class distribution for test: (array([1., 2.]), array([1492, 1508]))


In [4]:
from sklearn.svm import SVC

svm = SVC(verbose=True, kernel='poly', degree=3, C=1, random_state=42, decision_function_shape='ovr', probability=True)
svm.fit(x_train, y_train)

[LibSVM]................................................................................................................................*................................................*......................................................................................................................................*........................................................................................................................................................................*..............................................................................................................................................................................................................................*...............................................................................................................................................................................................................................................................................................

SVC(C=1, kernel='poly', probability=True, random_state=42, verbose=True)

In [5]:
print('accuracy:', svm.score(x_test, y_test))

from sklearn.metrics import classification_report, confusion_matrix
y_pred = svm.predict(x_test)

utils.present_confusion_matrix(confusion_matrix(y_test, y_pred))
utils.present_metrics(classification_report(y_test, y_pred, target_names=STATES, output_dict=True))

accuracy: 0.7246666666666667
Confusion Matrix:
+-----------------+--------------------+--------------------+
|                 | Predicted: boredom | Predicted: neutral |
+-----------------+--------------------+--------------------+
| Actual: boredom |        1223        |        269         |
| Actual: neutral |        557         |        951         |
+-----------------+--------------------+--------------------+

Classification Report:
+--------------+-----------+--------+----------+---------+
|              | precision | recall | f1-score | support |
+--------------+-----------+--------+----------+---------+
|   boredom    |   0.687   |  0.82  |  0.748   | 1492.0  |
|   neutral    |   0.78    | 0.631  |  0.697   | 1508.0  |
|   accuracy   |   0.725   | 0.725  |  0.725   |  0.725  |
|  macro avg   |   0.733   | 0.725  |  0.722   | 3000.0  |
| weighted avg |   0.734   | 0.725  |  0.722   | 3000.0  |
+--------------+-----------+--------+----------+---------+


In [6]:
from skl2onnx import to_onnx
onx = to_onnx(svm, x_train[:1].astype(np.float32), target_opset=12)
with open("./exported_models/tetris-boredom-neutral.onnx", "wb") as f:
    f.write(onx.SerializeToString())